# 2D Bicopter (Quadcopter)

We consider the 2D (vertical $z$ and horizontal axis $y$) quadcopter problem, and consider the following dynamics


State: $$[y, z, v_y, v_z]$$

Near hover Dynamics:
$$\begin{bmatrix}\dot y \\ \dot z \\ \dot v_y\\ \dot v_z\end{bmatrix} = \begin{bmatrix}v_x \\ v_y \\ -T \sin(\phi) \\ T \cos(\phi) - g\end{bmatrix}, \text{with } u=[\phi, T]$$


Approximation of near hover Dynamics: 
$$\begin{bmatrix}\dot y \\ \dot z \\ \dot v_y \\ \dot v_z\end{bmatrix}=\dot{X}=\begin{bmatrix}v_y \\ v_z \\-g\tan(\phi) \\ T-g\end{bmatrix}=\begin{bmatrix}v_y \\ v_z \\ -gu_1 \\ u_2 - g\end{bmatrix}, \text{with } u=[\tan(\phi), T]$$

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.version

'3.9.18 | packaged by conda-forge | (main, Aug 30 2023, 03:49:32) \n[GCC 12.3.0]'

In [3]:
import matplotlib
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.size': '20',
    'text.usetex': False,   # Toggle to true for official LaTeX output
    'pgf.rcfonts': False,
    'lines.linewidth': 4.,
})

In [4]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
from matplotlib import MatplotlibDeprecationWarning
warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning)
import matplotlib.pyplot as plt
import jax.numpy as jnp
import cvxpy as cp
import jax
import numpy as np
import seaborn as sns

import matplotlib
import pickle as pkl
import pandas as pd

from scipy.interpolate import interp1d
from matplotlib.animation import FuncAnimation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

In [5]:
import hj_reachability as hj
import cbf_opt
from cbf_opt import ControlAffineDynamics, ControlAffineCBF, ControlAffineASIF, SlackifiedControlAffineASIF, BatchedDynamics
from experiment_wrapper import RolloutTrajectory, StateSpaceExperiment, TimeSeriesExperiment

from refine_cbfs import HJControlAffineDynamics, TabularControlAffineCBF, TabularTVControlAffineCBF, utils

from quad_2d.animate_quad import animate_multi_planar_quad, get_drone

## Setup Problem (dynamics, environment and CBF)

### Dynamics

In [6]:
class CrazyflieDynamics(ControlAffineDynamics):
    """
    Simplified dynamics, and we need to convert controls from phi to tan(phi)"""
    STATES = ["y", "z", "v_y", "v_z", "nu"]  # added dim
    CONTROLS = ["tan(phi)", "T"]
    DISTURBANCES = ["dy"]
    def __init__(self, params, test=True, **kwargs):
        super().__init__(params, test, **kwargs)
    
    def open_loop_dynamics(self, state, time: float = 0.0):
        return jnp.array([state[2], state[3], 0.0, -self.params['g'], 0.0]) # added dim

    def control_matrix(self, state, time: float = 0.0):
        return jnp.array([[0.0, 0.0], [0.0, 0.0], [9.81, 0.0], [0.0, 1.0], [0.0, 0.0]])  # added dim

    # def disturbance_matrix(self, state, time: float = 0.0):
    #     return jnp.array([[1.0, 0.0, 0.0, 0.0]]).reshape(len(self.STATES), len(self.DISTURBANCES))

    def state_jacobian(self, state, control, disturbance = None, time: float = 0.0):
        return jax.jacfwd(lambda x: self.__call__(x, control, disturbance, time))(state)
    

In [20]:
dyn = CrazyflieDynamics(params={'l': 8.0, 'g': [6., 9.81]}, dt=0.01, test=False)
batched_dyn = BatchedDynamics(dyn)
umax = jnp.array([jnp.tan(np.pi / 6), 1.1 * 9.81])
umin = jnp.array([-jnp.tan(np.pi / 6), 0.0])
# dmax = jnp.array([3.0])
# dmin = jnp.array([-3.0])

In [21]:
dyn.all_params

{'l': 8.0, 'g': [6.0, 9.81]}

In [22]:
dyn.get_param_combinations()

[{'l': 8.0, 'g': 6.0}, {'l': 8.0, 'g': 9.81}]

In [23]:
dyn.step(np.zeros(4), np.zeros(2))

AssertionError: All params must be fixed for simulation

In [9]:
dyn_alt = CrazyflieDiffDynamics(params={}, dt=0.01, test=False)
batched_dyn_alt = BatchedDynamics(dyn_alt)
umax_alt = jnp.array([jnp.tan(np.pi / 6), 1.1 * 9.81])
umin_alt = jnp.array([-jnp.tan(np.pi / 6), 0.0])
dmax_alt = jnp.array([9.81])
dmin_alt = jnp.array([6.0])

## Environment:
### Boundary of grid
Boundary of grid is defined below by the state domain
### Environment / obstacles
Safe set is delimited by the state space boundary and by obstacles

In [10]:
state_domain = hj.sets.Box(lo=jnp.array([-6., -0.1, -5., -5., 1.0]), 
                           hi=jnp.array([6., 4.1, 5., 5., 2.0]))  # added dim
grid_resolution = (61, 61, 31, 31, 5)  # added dim
grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(state_domain, grid_resolution)

In [11]:
boundary = np.array([[-10., 10.], [-4., 8.], [-10., 10.], [-10., 10.], [1.0, 2.0]])  # added dim (can ignore for inv ped)

In [12]:
obstacle1 = np.array([[-2., 0.], [1., 3.], [-100., 100.], [-100., 100.], [-100., 100.]])  # added dim
obstacles = [obstacle1]

In [13]:
sdf = utils.build_sdf(boundary, obstacles)

In [14]:
sdf_values = hj.utils.multivmap(sdf, jnp.arange(grid.ndim))(grid.states)

# Vanilla Reachability is used here

We consider here the comparison of a method that accounts for disturbances and one that doesn't

In [15]:
backwards_reachable_tube = lambda obstacle: (lambda t, x: jnp.minimum(x, obstacle))
solver_settings = hj.SolverSettings.with_accuracy("very_high", value_postprocessor=backwards_reachable_tube(sdf_values))
init_values = sdf_values
initial_time = 0.
final_time = -5.0
times = jnp.linspace(initial_time, final_time, 51)

### Disturbances

In [16]:
dyn_hjr = HJControlAffineDynamics(dyn, control_space=hj.sets.Box(umin, umax))
target_values = utils.hj_solve(solver_settings, dyn_hjr, grid, times, init_values)

 20%|#9        |  1.0000/5.0 [02:37<10:57, 164.36s/sim_s]

XlaRuntimeError: INTERNAL: Generated function failed: CpuCallback error: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /opt/anaconda3/envs/ros_noetic/lib/python3.9/site-packages/jax/_src/interpreters/mlir.py(2326): _wrapped_callback
  /opt/anaconda3/envs/ros_noetic/lib/python3.9/site-packages/jax/_src/interpreters/pxla.py(1144): __call__
  /opt/anaconda3/envs/ros_noetic/lib/python3.9/site-packages/jax/_src/profiler.py(314): wrapper
  /opt/anaconda3/envs/ros_noetic/lib/python3.9/site-packages/jax/_src/pjit.py(1152): _pjit_call_impl_python
  /opt/anaconda3/envs/ros_noetic/lib/python3.9/site-packages/jax/_src/pjit.py(1196): call_impl_cache_miss
  /opt/anaconda3/envs/ros_noetic/lib/python3.9/site-packages/jax/_src/pjit.py(1212): _pjit_call_impl
  /opt/anaconda3/envs/ros_noetic/lib/python3.9/site-packages/jax/_src/core.py(869): process_primitive
  /opt/anaconda3/envs/ros_noetic/lib/python3.9/site-packages/jax/_src/core.py(389): bind_with_trace
  /opt/anaconda3/envs/ros_noetic/lib/python3.9/site

In [ ]:
(target_values[-1] >= 0).sum() / target_values[-1].size

Array(0.9452462, dtype=float32)

In [ ]:
dyn2 = CrazyflieDynamics(params={'g': 9.81}, dt=0.01, test=False)
dyn_hjr2 = HJControlAffineDynamics(dyn2, control_space=hj.sets.Box(umin, umax))
target_values2 = utils.hj_solve(solver_settings, dyn_hjr2, grid, times, init_values)

100%|##########|  5.0000/5.0 [15:16<00:00, 183.39s/sim_s]


In [ ]:
(target_values2[-1] >= 0).sum() / target_values2[-1].size

Array(0.9500945, dtype=float32)

In [ ]:
dyn3 = CrazyflieDynamics(params={'g': 6.}, dt=0.01, test=False)
dyn_hjr3 = HJControlAffineDynamics(dyn3, control_space=hj.sets.Box(umin, umax))
target_values3 = utils.hj_solve(solver_settings, dyn_hjr3, grid, times, init_values)

100%|##########|  5.0000/5.0 [14:09<00:00, 169.92s/sim_s]


In [ ]:
(target_values3[-1] >= 0).sum() / target_values3[-1].size

Array(0.9530203, dtype=float32)

In [ ]:
dyn_hjr_alt = HJControlAffineDynamics(dyn_alt, control_space=hj.sets.Box(umin_alt, umax_alt), disturbance_space=hj.sets.Box(dmin_alt, dmax_alt))
target_values_alt = utils.hj_solve(solver_settings, dyn_hjr_alt, grid, times, init_values)

100%|##########|  5.0000/5.0 [15:27<00:00, 185.40s/sim_s]


In [ ]:
(target_values_alt[-1] >= 0).sum() / target_values_alt[-1].size

Array(0.9479823, dtype=float32)

In [ ]:
# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111)

div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')

vy_slice = grid.shape[2] // 2
vz_slice = grid.shape[3] // 2 - 5

vmax = np.abs(target_values[0]).max()
cf = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
                 target_values[0][:, :, vy_slice, vz_slice].T, vmax=vmax, vmin=-vmax)
cont_sdf = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           sdf_values[:, :, vy_slice, vz_slice].T, levels=[0], colors='k', linewidths=4)
cont = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           target_values[0][:, :, vy_slice, vz_slice].T, levels=[0], colors='green', linewidths=2)
cb = fig.colorbar(cf, cax=cax)
tx = ax.set_title(f'HJR time $t=0$')
ax.set_xlabel('$y$ (Horizontal)')
ax.set_ylabel('$z$ (Vertical)')
tx = ax.set_title(f'$v_y=0, v_z=0$, HJR time $t=0$')

# Update function to draw contours for a given idi value
def update(idi):
    global cont
    arr = target_values[idi][:, :, vy_slice, vz_slice].T
    vmax = np.abs(arr).max()
    # ax.clear()
    cf = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], arr, vmax=vmax, vmin=-vmax)
    cont.collections[0].remove()
    cont = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
               arr, levels=[0], colors='green')
    cax.cla()
    fig.colorbar(cf, cax=cax)
    tx.set_text('HJR time t={:.2f}'.format(np.abs(times[idi].item())))


# Animate with idi values from 0 to 11
ani = FuncAnimation(fig, update, frames=range(len(times)))
plt.close()
HTML(ani.to_jshtml())

In [ ]:
# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111)

div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')

vy_slice = grid.shape[2] // 2
vz_slice = grid.shape[3] // 2 - 5

vmax = np.abs(target_values[0]).max()
cf = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
                 target_values2[0][:, :, vy_slice, vz_slice].T, vmax=vmax, vmin=-vmax)
cont_sdf = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           sdf_values[:, :, vy_slice, vz_slice].T, levels=[0], colors='k', linewidths=4)
cont = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           target_values2[0][:, :, vy_slice, vz_slice].T, levels=[0], colors='green', linewidths=2)
cb = fig.colorbar(cf, cax=cax)
tx = ax.set_title(f'HJR time $t=0$')
ax.set_xlabel('$y$ (Horizontal)')
ax.set_ylabel('$z$ (Vertical)')
tx = ax.set_title(f'$v_y=0, v_z=0$, HJR time $t=0$')

# Update function to draw contours for a given idi value
def update(idi):
    global cont
    arr = target_values2[idi][:, :, vy_slice, vz_slice].T
    vmax = np.abs(arr).max()
    # ax.clear()
    cf = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], arr, vmax=vmax, vmin=-vmax)
    cont.collections[0].remove()
    cont = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
               arr, levels=[0], colors='green')
    cax.cla()
    fig.colorbar(cf, cax=cax)
    tx.set_text('HJR time t={:.2f}'.format(np.abs(times[idi].item())))


# Animate with idi values from 0 to 11
ani = FuncAnimation(fig, update, frames=range(len(times)))
plt.close()
HTML(ani.to_jshtml())

In [ ]:
# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111)

div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')

vy_slice = grid.shape[2] // 2
vz_slice = grid.shape[3] // 2 - 5

vmax = np.abs(target_values[0]).max()
cf = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
                 target_values3[0][:, :, vy_slice, vz_slice].T, vmax=vmax, vmin=-vmax)
cont_sdf = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           sdf_values[:, :, vy_slice, vz_slice].T, levels=[0], colors='k', linewidths=4)
cont = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           target_values3[0][:, :, vy_slice, vz_slice].T, levels=[0], colors='green', linewidths=2)
cb = fig.colorbar(cf, cax=cax)
tx = ax.set_title(f'HJR time $t=0$')
ax.set_xlabel('$y$ (Horizontal)')
ax.set_ylabel('$z$ (Vertical)')
tx = ax.set_title(f'$v_y=0, v_z=0$, HJR time $t=0$')

# Update function to draw contours for a given idi value
def update(idi):
    global cont
    arr = target_values3[idi][:, :, vy_slice, vz_slice].T
    vmax = np.abs(arr).max()
    # ax.clear()
    cf = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], arr, vmax=vmax, vmin=-vmax)
    cont.collections[0].remove()
    cont = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
               arr, levels=[0], colors='green')
    cax.cla()
    fig.colorbar(cf, cax=cax)
    tx.set_text('HJR time t={:.2f}'.format(np.abs(times[idi].item())))


# Animate with idi values from 0 to 11
ani = FuncAnimation(fig, update, frames=range(len(times)))
plt.close()
HTML(ani.to_jshtml())

In [ ]:
# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111)

div = make_axes_locatable(ax)
cax = div.append_axes('right', '5%', '5%')

vmax = np.abs(target_values[0]).max()
cf = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
                 target_values_no_d[0][:, :, grid.shape[2] // 2, grid.shape[3] // 2].T, vmax=vmax, vmin=-vmax)
cont_sdf = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           sdf_values[:, :, grid.shape[2] // 2, grid.shape[3] // 2].T, levels=[0], colors='k', linewidths=4)
cont = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           target_values_no_d[0][:, :, grid.shape[2] // 2, grid.shape[3] // 2].T, levels=[0], colors='green', linewidths=2)
cb = fig.colorbar(cf, cax=cax)
tx = ax.set_title(f'HJR time $t=0$')
ax.set_xlabel('$y$ (Horizontal)')
ax.set_ylabel('$z$ (Vertical)')
tx = ax.set_title(f'$v_y=0, v_z=0$, HJR time $t=0$')

# Update function to draw contours for a given idi value
def update(idi):
    global cont
    arr = target_values_no_d[idi][:, :, grid.shape[2] // 2, grid.shape[3] // 2].T
    vmax = np.abs(arr).max()
    # ax.clear()
    cf = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], arr, vmax=vmax, vmin=-vmax)
    cont.collections[0].remove()
    cont = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
               arr, levels=[0], colors='green')
    cax.cla()
    fig.colorbar(cf, cax=cax)
    tx.set_text('HJR time t={:.2f}'.format(np.abs(times[idi].item())))


# Animate with idi values from 0 to 11
ani = FuncAnimation(fig, update, frames=range(len(times)))
plt.close()
HTML(ani.to_jshtml())

## Online implementation

### Comparisons
We compare the following algorithms:
- Nominal safety-agnostic control
- Heuristic CBF (what people use in practice!)
- CBVF computed using `refineCBF` from heuristic CBF offline
- Time varying CBVF (Ours)

### Value function evolution

The timescales over which we solved for the value function offline (see above) might not be real-time feasible. Hence we consider it is solved at a different frequency (e.g., artificially slowing down how the value function changes), to test out the framework.

Here we "slow" the reachability updates by a factor 4!

In [ ]:
tabular_finalized_cbf = TabularControlAffineCBF(batched_dyn, {}, test=False, grid=grid)
tabular_finalized_cbf.vf_table = target_values[-1]  # Take the last value (when converged)

In [ ]:
tcbf_no_d = TabularControlAffineCBF(batched_dyn, {}, test=False, grid=grid)
tcbf_no_d.vf_table = target_values_no_d[-1]  # Take the last value (when converged)

### Nominal control
We consider an LQR controller for nominal control. If you modify the experiment you can see that an LQR controller with safety filter is safe but can lead to us getting stuck

In [ ]:
x_nom = jnp.array([0.0, 3.0, 0.0, 0.0])
u_nom = jnp.array([0.0, 9.81])
A, B, _ = dyn.linearized_dt_dynamics(x_nom, u_nom)  # For discrete LQR!

Q = jnp.diag(jnp.array([1.0, 1.0, 0.1, 0.1]))
R = jnp.diag(jnp.array([1.0, 1.0]))

K = cbf_opt.utils.lqr(A, B, Q, R)

A_cl = A - B @ K
# assert np.all(np.linalg.eigvals(A_cl) < 0).all() 

In [ ]:
nominal_control = lambda u_ref, x_ref, F: lambda x, t: np.atleast_2d(np.clip(u_ref - 
                                (F @ (x - x_ref).T).T, umin, umax))

### Safety filter
We use the slackified version of the safety filter to ensure , we use gurobi as a solver (you need to obtain an academic license, or change the solver to e.g. "ECOS" / "OSQP")

In [ ]:
x_goal = jnp.array([4.5, 1.0, 0.0, 0.0])
u_goal = jnp.array([0.0, 9.81])
nom_control = nominal_control(u_goal, x_goal, K)
alpha = lambda x: 3.0 * x

cbvf_asif = SlackifiedControlAffineASIF(batched_dyn, tabular_finalized_cbf, test=False, alpha=alpha, nominal_policy=nom_control,
                                        umin=umin, umax=umax, dmin=dmin, dmax=dmax, solver=cp.GUROBI)

cbvf_asif_no_d = SlackifiedControlAffineASIF(batched_dyn, tcbf_no_d, test=False, alpha=alpha, nominal_policy=nom_control,
                                             umin=umin, umax=umax, solver=cp.GUROBI)

In [ ]:
x0 = jnp.array([-4.0, 2.0, 0.0, 0.0])
experiment = RolloutTrajectory('quad', start_x=x0, n_sims_per_start=1, t_sim=10.0)

In [ ]:
class UniformDisturbance:
    def __init__(self, bounds, **kwargs):
        self.bounds = bounds
        self.seed = kwargs.get("seed", 0)
        self.beta_skew = kwargs.get("beta_skew", 1.0)  # Defaults to a uniform distribution (beta(1,1) = uniform(0,1))
        self.reset()
    
    def __call__(self, x, t):
        # Randomized value
        return self.random_state.beta(self.beta_skew, self.beta_skew, size=(x.shape[0], self.bounds.shape[0])) * (self.bounds[:, 1] - self.bounds[:, 0]) + self.bounds[:, 0]
    
    def reset(self, x=None):
        # Resets random state to have same disturbances for each rollout
        self.random_state = np.random.default_rng(seed=self.seed)

In [ ]:
dist = UniformDisturbance(np.array([dmin, dmax]).T)

In [ ]:
import logging
logging.getLogger('cbf_opt').setLevel(level=logging.ERROR)
results_df = experiment.run(batched_dyn, {"nominal": nom_control, "Disturbance aware CBF": cbvf_asif, "Nonaware CBF": cbvf_asif_no_d}, disturbances={"uniform": dist})

In [ ]:
# Find closest time in results_df.t to time_ind
#### NO NEED TO MODIFY TYPICALLY ####
def find_closest_time(df, time_ind):
    return df.t.iloc[df.t.sub(time_ind).abs().idxmin()]


ss_exp = StateSpaceExperiment('quad', x_indices=[0, 1], start_x=x0)
# Set up the figure and axis
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
alphas = [0.1, 0.5, 0.5, 1.0]
nbr_controllers = len(results_df.controller.unique())

plt.legend(results_df.controller.unique())
ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           tabular_finalized_cbf.vf_table[:, :, grid.shape[2] // 2, grid.shape[3] // 2].T, levels=[0], colors='grey', linewidths=4)  
cont3 = ax.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           sdf_values[:, :, grid.shape[2] // 2, grid.shape[3] // 2].T, levels=[-10, 0], colors='red')
cont2 = ax.contour(grid.coordinate_vectors[0], grid.coordinate_vectors[1],
           sdf_values[:, :, grid.shape[2] // 2, grid.shape[3] // 2].T, levels=[0], colors='k', linewidths=4)
ax.set_xlabel('$y$ (Horizontal)', fontsize=20)
ax.set_ylabel('$z$ (Vertical)', fontsize=20)
tx = ax.set_title('$v_y=0, v_z=0$, HJR time $t=0$')
ss_exp.plot(batched_dyn, results_df, ax=ax, add_direction=False, max_time=0.0, alpha=alphas)
ax.legend(ax.lines[::len(ax.lines) // nbr_controllers], results_df.controller.unique(), loc="upper center", ncol=4, fontsize=20)

# Update function to draw contours for a given idi value
def update(time):
    for line in ax.lines:
        line.remove()
    for patch in ax.patches:
        patch.remove()
    ax.set_prop_cycle(None)
    ss_exp.plot(batched_dyn, results_df, ax=ax, add_direction=False, max_time=time, alpha=alphas)
    closest_time = find_closest_time(results_df, time)
    curr_vals = results_df[(results_df.t == closest_time) & (results_df.measurement.isin(["y", "z", "tan(phi)"]))].value.values.reshape(nbr_controllers, -1)
    colors = []
    for line in ax.lines[::len(ax.lines) // nbr_controllers]:
        colors.append(line.get_color())
    for i, curr_val in enumerate(curr_vals):
        # get color from prop_cycle 
        get_drone(ax, curr_val[0], curr_val[1], np.arctan(-curr_val[2]), rel_size=0.3, height_scale=0.9, alpha=alphas[i], color=colors[i])
    tx.set_text('$v_y=0, v_z=0$, Simulation time t={:.2f}'.format(np.abs(time)))
    fig.tight_layout()

ani = FuncAnimation(fig, update, frames=np.linspace(0,10,100))
plt.close()
HTML(ani.to_jshtml())

In [ ]:
ts_exp = TimeSeriesExperiment('quad', x_indices=[0, 1], start_x=x0)

In [ ]:
fighandle = ts_exp.plot(batched_dyn, results_df, extra_measurements=['vf'])[0]
fig = fighandle[1]
ax = fig.axes[-1]
ax.plot(np.arange(0,20), np.zeros_like(np.arange(0,20)), 'k--', linewidth=4)